### Sample spatial means for ACT OCSE project

In [1]:
from google.cloud import storage
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import warnings
import os
from io import StringIO, BytesIO
import xarray as xr
import requests
import json
from pyproj import Geod
from datetime import datetime, timedelta
from scipy.spatial import cKDTree
from itertools import product

In [ ]:
gsutil -m cp -r \
  "gs://terrakio-mass-requests/KG7zkkvurzSh23Ue70C5k7groNy2/ACT23_5km/data/ver1/*" \
  /home/muye/aus-env-modis/OCSE/lc23_byCells


gsutil -m cp -r \
  "gs://terrakio-mass-requests/KG7zkkvurzSh23Ue70C5k7groNy2/ACTVeg_2023_5km/data/ver1/*" \
  /home/muye/aus-env-modis/OCSE/vegCode_byCells

### step 3. intersect cells with polygon

---

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
import os
import rioxarray
from shapely.geometry import mapping

def process_cell(cell_id, polygons_gdf, landcover_nc, vegtype_nc, output_dir):
    """
    Process a single cell for all districts and vegetation types.
    
    Parameters:
    -----------
    cell_polygon : geopandas.GeoDataFrame
        The current cell geometry
    district_polygons : geopandas.GeoDataFrame
        All district polygons
    landcover_nc : str
        Path to landcover netCDF file
    vegtype_nc : str
        Path to vegetation type netCDF file
    output_dir : str
        Directory to save the output tables
    """
    # Load netCDF files
    landcover_ds = xr.open_dataset(landcover_nc)
    landcover_ds = landcover_ds.isel(time=0)
    vegtype_ds = xr.open_dataset(vegtype_nc)
    vegtype_ds = vegtype_ds.isel(time=0)
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Assign CRS to both datasets
    crs = 'EPSG:32755'
    landcover_ds.rio.write_crs(crs, inplace=True)
    vegtype_ds.rio.write_crs(crs, inplace=True)
    
    results = []
    
    # Process each district
    polygons_pdf = polygons_gdf.set_crs('EPSG:4326')
    polygons_transformed = polygons_gdf.to_crs('EPSG:32755')

    for idx, polygon in polygons_transformed.iterrows():
        polygon_name = polygon['DIVISION_N']

        geom = [mapping(polygon.geometry)]    
        
        # Mask data for current polygon with improved clipping parameters
        landcover_masked = landcover_ds.rio.clip(geom, drop=False, all_touched=True)
        vegtype_masked = vegtype_ds.rio.clip(geom, drop=False, all_touched=True)
        
        # Get unique vegetation types in the masked area
        veg_types = np.unique(vegtype_masked['var0'].values)
        veg_types = veg_types[~np.isnan(veg_types)]


        # Process each vegetation type
        for veg_type in veg_types:
            # Create vegetation type mask
            veg_mask = (vegtype_masked['var0'].values == veg_type)
            
            # Process each land cover class
            for class_idx in range(7):
                class_data = landcover_masked[f'var{class_idx}'].values
                
                # Apply vegetation type mask on land cover classes
                masked_data = np.where(veg_mask, class_data, np.nan)
                
                # Calculate metrics
                valid_pixels = np.count_nonzero(~np.isnan(masked_data))  # after mask, non-nans are pixels of this veg type
                mean_prob = np.nanmean(masked_data)
                
                # Store results
                results.append({
                    'polygon': polygon_name,
                    'vegetation_type': int(veg_type),
                    'landcover_class': class_idx,
                    'pixel_count': valid_pixels,
                    'mean_probability': mean_prob
                })
    
    # Create and save table
    if results:
        df = pd.DataFrame(results)
        output_file = os.path.join(output_dir, f'cell_{cell_id}.csv')
        df.to_csv(output_file, index=False)
        return df
    return None



def process_all_cells(cell_ids, polygons_path, base_lc_path, base_veg_path, output_dir='celltables'):
    """Process all cells and save individual CSV files."""
    
    # Load polygons
    polygons_gdf = gpd.read_file(polygons_path)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    all_results = []
    
    # Process each cell
    for cell_id in cell_ids:
        # Construct paths
        landcover_nc= Path(base_lc_path) / f"{cell_id}.nc"
        vegtype_nc = Path(base_veg_path) / f"{cell_id}.nc"
        
        process_cell(cell_id, polygons_gdf, landcover_nc, vegtype_nc, output_dir)
                
    
   

# Define paths (update these as needed)
base_dir = '/home/muye/aus-env-modis/OCSE'
output_dir = os.path.join(base_dir, 'celltables_by_division23')
#polygons_path = '/home/muye/aus-env-modis/OCSE/actgov_districts.geojson'
polygons_path = '/home/muye/aus-env-modis/OCSE/actgov_divsion_full.geojson'
base_lc_path = '/home/muye/aus-env-modis/OCSE/lc23_byCells'
base_veg_path = '/home/muye/aus-env-modis/OCSE/vegCode_byCells'

cell_ids = np.arange(1, 42)
process_all_cells(cell_ids, polygons_path, base_lc_path, base_veg_path, output_dir='celltables_by_division23')


#### only a subset of cells (ACT scale) will intersect the divisions 

---


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import os

def aggregate_polygon_data(cell_tables_dir, output_dir):
    """
    Aggregate cell-level data into polygon-level tables with weighted probabilities.
    
    Parameters:
    -----------
    cell_tables_dir : str
        Directory containing all cell-level tables
    output_dir : str
        Directory to save polygon-level aggregated tables
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Read all cell tables
    cell_tables = []
    for table_path in glob.glob(os.path.join(cell_tables_dir, "*.csv")):
        df = pd.read_csv(table_path)
        cell_tables.append(df)
    
    # Combine all cell tables
    combined_df = pd.concat(cell_tables, ignore_index=True)
    
    # Get unique polygon (district) names
    unique_polygons = combined_df['polygon'].unique()
    
    # Process each polygon
    for polygon_name in unique_polygons:
        print(f"\nProcessing polygon {polygon_name}")
        
        # Filter data for current polygon
        polygon_data = combined_df[combined_df['polygon'] == polygon_name]
        
        results = []
        
        # Group by vegetation type and land cover class
        groups = polygon_data.groupby(['vegetation_type', 'landcover_class'])
        
        for (veg_type, lc_class), group in groups:
            # Calculate weighted sum (numerator)
            weighted_sum = np.sum(group['pixel_count'] * group['mean_probability'])
            
            # Calculate total pixel count (denominator)
            total_pixels = np.sum(group['pixel_count'])
            
            # Calculate weighted probability
            weighted_prob = weighted_sum / total_pixels if total_pixels > 0 else 0
            
            results.append({
                'vegetation_type': veg_type,
                'landcover_class': lc_class,
                'total_pixels': total_pixels,
                'weighted_probability': weighted_prob
            })
        
        # Create and save polygon table
        if results:
            polygon_df = pd.DataFrame(results)
            
            # Sort for better readability
            polygon_df = polygon_df.sort_values(['vegetation_type', 'landcover_class'])
            
            # Save table
            output_file = os.path.join(output_dir, f'{polygon_name.replace(" ", "_")}_aggregated.csv')
            polygon_df.to_csv(output_file, index=False)


# Define directories
base_dir = '/home/muye/aus-env-modis/OCSE'
cell_tables_dir = os.path.join(base_dir, 'celltables_by_division23')
output_dir = os.path.join(base_dir, 'polygon_tables_division23')

# Process the aggregation
aggregate_polygon_data(cell_tables_dir, output_dir)




Processing polygon BANKS

Processing polygon CALWELL

Processing polygon CONDER

Processing polygon THEODORE

Processing polygon AMAROO

Processing polygon GUNGAHLIN

Processing polygon NGUNNAWAL

Processing polygon NICHOLLS

Processing polygon TAYLOR

Processing polygon MONCRIEFF

Processing polygon CASEY

Processing polygon FORDE

Processing polygon BONNER

Processing polygon JACKA

Processing polygon CHARNWOOD

Processing polygon DUNLOP

Processing polygon FRASER

Processing polygon HIGGINS

Processing polygon HOLT

Processing polygon LATHAM

Processing polygon MACGREGOR

Processing polygon STRATHNAIRN

Processing polygon MACNAMARA

Processing polygon HUME

Processing polygon OAKS ESTATE

Processing polygon PIALLIGO

Processing polygon BEARD

Processing polygon HAWKER

Processing polygon SCULLIN

Processing polygon WHITLAM

Processing polygon HALL

Processing polygon MITCHELL

Processing polygon GIRALANG

Processing polygon KALEEN

Processing polygon LAWSON

Processing polygon PALM

In [4]:
import pandas as pd
import glob
import os
from pathlib import Path

def combine_polygon_tables(polygon_tables_dir, output_file):
    """
    Combine all polygon tables into a single CSV file.
    
    Parameters:
    -----------
    polygon_tables_dir : str
        Directory containing all polygon-level tables
    output_file : str
        Path to the output combined CSV file
    """
    # Get all polygon table files
    table_files = glob.glob(os.path.join(polygon_tables_dir, '*_aggregated.csv'))
    
    # List to store all dataframes
    dfs = []
    
    # Process each polygon table
    for file_path in table_files:
        # Read the table
        df = pd.read_csv(file_path)
        
        # Extract polygon name from filename
        # Remove '_aggregated.csv' and get the base name
        polygon_name = Path(file_path).stem.replace('_aggregated', '')
        
        # Add polygon name as a column
        df['polygon_name'] = polygon_name
        
        dfs.append(df)
    
    # Combine all dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Reorder columns to put polygon_name first
    columns_order = ['polygon_name', 'vegetation_type', 'landcover_class', 
                    'total_pixels', 'weighted_probability']
    combined_df = combined_df[columns_order]
    
    # Sort by polygon_name, vegetation_type, and landcover_class
    combined_df = combined_df.sort_values(['polygon_name', 'vegetation_type', 'landcover_class'])
    
    # Save to CSV
    combined_df.to_csv(output_file, index=False)
    
    return combined_df


# Define directories and output file
base_dir = '/home/muye/aus-env-modis/OCSE'
polygon_tables_dir = os.path.join(base_dir, 'polygon_tables_division23')
output_file = os.path.join(base_dir, 'lc23_byDivision.csv')

# Combine tables
combined_df = combine_polygon_tables(polygon_tables_dir, output_file)



In [11]:
df = pd.read_csv('/home/muye/aus-env-modis/OCSE/lc23_byDivision.csv')
df.head()

,polygon_name,vegetation_type,landcover_class,total_pixels,weighted_probability
0,ACTON,381,0,1546,0.000000
1,ACTON,381,1,1546,0.036025
2,ACTON,381,2,1546,0.001011
3,ACTON,381,3,1546,0.167204
4,ACTON,381,4,1546,0.070390


In [9]:
df[df['landcover_class'] == 0]['total_pixels'].sum() * 100 / 10000
# ACT: 2358 sqkm2.

76709.23

In [13]:
df[(df['polygon_name'] == 'WANNIASSA') & (df['landcover_class'] == 0)]['total_pixels'].sum() * 100 / 10000   #1.5km^2

549.36